**Install Requirements**

In [0]:
"""!pip3 install 'torch==1.4.0'
!pip3 install 'torchvision==0.5.0'
!pip3 install 'Pillow-SIMD'
!pip3 install 'tqdm'"""

In [2]:
import os
if not os.path.isdir('./Pacs'):
  !git clone https://github.com/lore-lml/machine-learning2020-hw3.git
  !mv 'machine-learning2020-hw3' 'Pacs'
  !rm './Pacs/hw3.ipynb'
  !rm './Pacs/README.md'

import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from pacs_dataset import Pacs
from dann import alexdann
from dann import train_src, test_target, dann_train_src_target

from PIL import Image
from tqdm import tqdm

from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
%matplotlib inline

Cloning into 'machine-learning2020-hw3'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 10072 (delta 26), reused 30 (delta 14), pack-reused 10029
Receiving objects: 100% (10072/10072), 175.14 MiB | 11.00 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Checking out files: 100% (9996/9996), done.


**Set Arguments**

In [0]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 128      # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                      # the batch size, learning rate should change by the same factor to have comparable results

LR = 1e-3             # The initial Learning Rate
MOMENTUM = 0.9        # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5   # Regularization, you can keep this at the default

NUM_EPOCHS = 10       # Total number of training epochs (iterations over dataset)
STEP_SIZE = 3         # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.5           # Multiplicative factor for learning rate step-down

ALPHA = 'dynamic'

MODEL = 'pytorch'
# MODEL = 'caffe'

param_grid = {
    "lr": [1e-3],
    "batch_size": [128],
    "epochs": [10],
    "step_size": [3],
    "gamma": [0.5],
    "alpha": [0.005, 0.05, 0.5]
}

BASE_FILE_PATH = "RUN_1_LR1e-3_DynAlpha_SGD_SS2_GAMMA03"

**Define Data Preprocessing**

In [0]:
# Mean caffe model [0.48109378 0.45752458 0.40787054]
transforms = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [5]:
ROOT = 'Pacs/PACS'

source_data = Pacs(ROOT, transform=transforms, source='photo')
cartoon_data = Pacs(ROOT, transform=transforms, source='cartoon')
sketch_data = Pacs(ROOT, transform=transforms, source='sketch')
target_data = Pacs(ROOT, transform=transforms, source='art_painting')

_, source_labels = source_data.get_img_with_labels()
_, cartoon_labels = cartoon_data.get_img_with_labels()
_, sketch_labels = sketch_data.get_img_with_labels()
_, target_labels = target_data.get_img_with_labels()

print(f"# classes source_data: {len(set(source_labels))}")
print(f"# classes cartoon_data: {len(set(cartoon_labels))}")
print(f"# classes sketch_data: {len(set(sketch_labels))}")
print(f"# classes target_data: {len(set(target_labels))}")
print(f"source_data: {len(source_data)} elements")
print(f"cartoon_data: {len(cartoon_data)} elements")
print(f"sketch_data: {len(sketch_data)} elements")
print(f"target_data: {len(target_data)} elements")

# classes source_data: 7
# classes cartoon_data: 7
# classes sketch_data: 7
# classes target_data: 7
source_data: 1670 elements
cartoon_data: 2344 elements
sketch_data: 3929 elements
target_data: 2048 elements


**Prepare Dataloaders**

In [0]:
def get_data_loaders(dataset, batch_size=BATCH_SIZE):
  return DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=True), DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)


**Prepare Network**

In [0]:
def getNet(pretrained=False, src=MODEL):
    return alexdann(pretrained, src=src)

def init_cnn_objects(model, lr=LR, step_size=STEP_SIZE, gamma=GAMMA):
  
  # Define loss function
  criterion_1 = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
  criterion_2 = nn.CrossEntropyLoss()
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
  
  optimizer = optim.SGD(parameters_to_optimize, lr=lr, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  #optimizer = optim.Adam(parameters_to_optimize, lr=lr,amsgrad=True)
  #optimizer = optim.AdamW(parameters_to_optimize, lr=lr,amsgrad=True, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

  return criterion_1, criterion_2, optimizer, scheduler

**Training**

In [0]:
def saveResults(configuration, best_mean_accuracy, accuracies_test, class_losses, domain_losses=None,
                dann=False, append=False, base_path='prova'):
  suffix_path = "_results.txt"
  mode = "a" if append else "w"
  with open(f"{base_path}{suffix_path}", mode, encoding='utf-8') as f:
      title = "********** WITHOUD DANN **********" if not dann else "********** WITH DANN **********"
      f.write(f"{title}\n")
      f.write(f"Best Configuration:\n{configuration}\n\n")
      f.write(f"Best Mean Accuracy: {best_mean_accuracy}\n")
      f.write(f"Accuracies on Target:\n")
      join = ",".join(list(map(lambda x: str(x), accuracies_test)))
      f.write(f"{join}\n")
      f.write(f"Best Accuracy on Target: {max(accuracies_test)}\n")
      f.write("Class Losses:\n")
      join = ",".join(list(map(lambda x: str(x), class_losses)))
      f.write(f"{join}\n")
      if dann and domain_losses is not None:
          join = ",".join(list(map(lambda x: str(x), domain_losses)))
          f.write(f"{join}\n")
      f.write("\n\n")

  print("********* FILE SAVED *********")

In [0]:
def simple_train_test(model, source_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=NUM_EPOCHS, file_path=BASE_FILE_PATH, device=DEVICE):
    train_losses = []
    accuracies = []
    loss_min = -1
    accuracy_max = 0
    
    model = model.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_lr()))
        cumulative_loss, current_step = train_src(model, source_dataloader, optimizer, criterion, current_step, device)
        curr_loss = cumulative_loss / len(source_dataloader)
        train_losses.append(curr_loss)
        """if loss_min == -1 or loss_min > curr_loss:
            loss_min = curr_loss
            torch.save(model, f"{file_path}_best_model.pth")"""
        curr_accuracy = test_target(model, test_dataloader, device) / float(len(target_data))
        accuracies.append(curr_accuracy)
        print(f"Current Accuracy: {curr_accuracy}")
        if accuracy_max < curr_accuracy:
          accuracy_max = curr_accuracy
          # torch.save(model, f"{file_path}_best_model.pth")
        scheduler.step()
        
    # model = torch.load(f"{file_path}_best_model.pth").to(device)
    # accuracy = test_target(model, test_dataloader, device) / float(len(target_data))
    # print(f"Accuracy on test set: {accuracy}%")
    return train_losses, accuracies

In [10]:
grid = ParameterGrid(param_grid)
results = []
for config in grid:
  source_dataloader, _ = get_data_loaders(source_data, config['batch_size'])
  _, cartoon_test_dataloader = get_data_loaders(cartoon_data, config['batch_size'])
  _, sketch_test_dataloader = get_data_loaders(sketch_data, config['batch_size'])
  no_dann_cartoon = getNet(pretrained=True)
  criterion_cartoon, _, optimizer_cartoon, scheduler_cartoon = init_cnn_objects(no_dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  no_dann_sketch = getNet(pretrained=True)
  criterion_sketch, _, optimizer_sketch, scheduler_sketch = init_cnn_objects(no_dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  cartoon_losses, cartoon_accuracies = simple_train_test(no_dann_cartoon, source_dataloader, cartoon_test_dataloader, 
                                                           criterion_cartoon, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'])
  sketch_losses, sketch_accuracies = simple_train_test(no_dann_sketch, source_dataloader, sketch_test_dataloader, 
                                                         criterion_sketch, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'])

  curr_result = {'params': config, 'cartoon_losses': cartoon_losses, 'sketch_losses': sketch_losses, 
                 'best_acc': np.mean([max(cartoon_accuracies), max(sketch_accuracies)])}
  results.append(curr_result)
  
best_conf = max(results, key=lambda x: x['best_acc'])
print(f"Best configuration is:\n{best_conf['params']}")
print(f"Highest mean accuracy: {best_conf['best_acc']}")


Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/checkpoints/alexnet-owt-4df8aa71.pth



Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss_train 1.9942938089370728
Step 10, Loss_train 0.3455280065536499


100%|██████████| 19/19 [00:05<00:00,  3.68it/s]

...

100%|██████████| 31/31 [00:08<00:00,  3.67it/s]

Current Accuracy: 0.390625
Starting epoch 10/10, LR = [6.25e-05]


Step 120, Loss_train 0.04150238260626793


100%|██████████| 31/31 [00:08<00:00,  3.67it/s]

Current Accuracy: 0.3974609375
Best configuration is:
{'alpha': 0.5, 'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.001, 'step_size': 3}
Highest mean accuracy: 0.38427734375


In [11]:
params = best_conf['params']

source_dataloader, _ = get_data_loaders(source_data, batch_size=params['batch_size'])
_, test_dataloader = get_data_loaders(target_data, batch_size=params['batch_size'])
no_dann = getNet(pretrained=True)
criterion, _, optimizer, scheduler = init_cnn_objects(no_dann, lr=params['lr'], step_size=params['step_size'], gamma=params['gamma'])

losses_no_dann, accuracies_no_dann = simple_train_test(no_dann, source_dataloader, test_dataloader, criterion, optimizer, scheduler, max_epoch=params['epochs'])
print(f"Photo transfer to Art accuracy WITHOUT DANN: {max(accuracies_no_dann)}")

saveResults(params, best_conf['best_acc'], accuracies_no_dann, losses_no_dann)

Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0, Loss_train 2.326894521713257
Step 10, Loss_train 0.44786256551742554


100%|██████████| 16/16 [00:04<00:00,  3.40it/s]

Current Accuracy: 0.4775390625
Starting epoch 2/10, LR = [0.001]


...

100%|██████████| 16/16 [00:04<00:00,  3.35it/s]

Current Accuracy: 0.4970703125
Starting epoch 10/10, LR = [6.25e-05]


Step 120, Loss_train 0.028234347701072693


100%|██████████| 16/16 [00:04<00:00,  3.41it/s]

Current Accuracy: 0.49560546875
Photo transfer to Art accuracy WITHOUT DANN: 0.50244140625
********* FILE SAVED *********


**Training with DANN**

In [0]:
import math
def dann_train_test(model, source_dataloader, target_dataloader, test_dataloader, 
                    class_criterion, domain_criterion, optimizer, scheduler, max_epoch=NUM_EPOCHS, alpha=ALPHA, file_path=BASE_FILE_PATH, device=DEVICE):
    class_losses_y = []
    domain_losses_d = []
    accuracies = []
    loss_min = -1
    accuracy_max = 0
    count_diverge = 0
    
    model = model.to(device)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(max_epoch):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, max_epoch, scheduler.get_lr()))
        class_loss, domain_loss, current_step = dann_train_src_target(model, source_dataloader, 
                                          target_dataloader, optimizer, class_criterion, domain_criterion, current_step,
                                          epoch, max_epoch, alpha=alpha, device=device)
        
        if math.isnan(class_loss) or math.isnan(domain_loss):
          count_diverge += 1
          if count_diverge >= 3:
            print("EARLY STOPPING")
            break;
        class_losses_y.append(class_loss)
        domain_losses_d.append(domain_loss)

        """if loss_min == -1 or loss_min > class_loss:
            loss_min = class_loss
            torch.save(model, f"{file_path}_best_model_dann.pth")"""
        accuracy = test_target(model, test_dataloader, device) / float(len(target_data))
        accuracies.append(accuracy)
        print(f"Accuracy on test set: {accuracy}")
        if accuracy_max < accuracy:
          accuracy_max = accuracy
          torch.save(model, f"{file_path}_best_model_dann.pth")
        scheduler.step()
        
    return class_losses_y, domain_losses_d, accuracies

In [13]:
grid = ParameterGrid(param_grid)
dann_results = []
for config in grid:
  source_dataloader, _ = get_data_loaders(source_data, config['batch_size'])
  cartoon_dataloader, cartoon_test_dataloader = get_data_loaders(cartoon_data, config['batch_size'])
  sketch_dataloader, sketch_test_dataloader = get_data_loaders(sketch_data, config['batch_size'])
  dann_cartoon = getNet(pretrained=True)
  criterion_cartoon_src, criterion_cartoon_tgt, optimizer_cartoon, scheduler_cartoon = init_cnn_objects(dann_cartoon, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])
  dann_sketch = getNet(pretrained=True)
  criterion_sketch_src, criterion_sketch_tgt, optimizer_sketch, scheduler_sketch = init_cnn_objects(dann_sketch, lr=config['lr'], 
                                                                          step_size=config['step_size'], gamma=config['gamma'])

  class_losses_cartoon, domain_losses_cartoon, accuracies_cartoon = dann_train_test(
      dann_cartoon, source_dataloader, cartoon_dataloader, cartoon_test_dataloader,
      criterion_cartoon_src, criterion_cartoon_tgt, optimizer_cartoon, scheduler_cartoon, max_epoch=config['epochs'], alpha=config['alpha']
  )

  class_losses_sketch, domain_losses_sketch, accuracies_sketch = dann_train_test(
      dann_sketch, source_dataloader, sketch_dataloader, sketch_test_dataloader,
      criterion_sketch_src, criterion_sketch_tgt, optimizer_sketch, scheduler_sketch, max_epoch=config['epochs'], alpha=config['alpha']
  )

  curr_result = {'params': config, 'cartoon_losses': (class_losses_cartoon, domain_losses_cartoon), 'sketch_losses': (class_losses_sketch, domain_losses_sketch),
                 'best_acc': np.mean([max(accuracies_cartoon), max(accuracies_sketch)])}
  dann_results.append(curr_result)
  
best_conf = max(dann_results, key=lambda x: x['best_acc'])
print(f"Best configuration is:\n{best_conf['params']}")
print(f"Highest mean accuracy: {best_conf['best_acc']}")


Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0
Class Loss 2.22733998298645, Domain Loss 1.5782082080841064
Step 10
Class Loss 0.4505959749221802, Domain Loss 0.15484854578971863


100%|██████████| 19/19 [00:05<00:00,  3.65it/s]


Accuracy on test set: 0.19873046875
Starting epoch 2/10, LR = [0.001]
Step 20
Class Loss 0.0795367881655693, Domain Loss 0.052016615867614746


...

Step 110
Class Loss 0.04328165575861931, Domain Loss 1.1548399925231934e-07


100%|██████████| 31/31 [00:08<00:00,  3.67it/s]


Accuracy on test set: 0.50390625
Starting epoch 10/10, LR = [6.25e-05]
Step 120
Class Loss 0.03218480572104454, Domain Loss 1.4901161193847656e-08


100%|██████████| 31/31 [00:08<00:00,  3.71it/s]


Accuracy on test set: 0.51220703125
Best configuration is:
{'alpha': 0.5, 'batch_size': 128, 'epochs': 10, 'gamma': 0.5, 'lr': 0.001, 'step_size': 3}
Highest mean accuracy: 0.560302734375


In [14]:
params = best_conf['params']

source_dataloader, _ = get_data_loaders(source_data, params['batch_size'])
target_dataloader, test_dataloader = get_data_loaders(target_data, params['batch_size'])

dann = getNet(pretrained=True)
class_criterion, domain_criterion, optimizer, scheduler = init_cnn_objects(dann, lr=params['lr'], step_size=params['step_size'],
                                                                           gamma=params['gamma'])
class_losses_y, domain_losses_d, accuracies_dann = dann_train_test(dann, source_dataloader, target_dataloader,
                          test_dataloader, class_criterion, domain_criterion, optimizer, scheduler, max_epoch=params['epochs'], alpha=params['alpha'])

print(f"Photo transfer to Art accuracy DANN: {max(accuracies_dann)}")
saveResults(params, best_conf['best_acc'], accuracies_dann, class_losses_y, domain_losses=domain_losses_d,
            dann=True, append=True)

Starting epoch 1/10, LR = [0.001]


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:351: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  "please use `get_last_lr()`.", UserWarning)


Step 0
Class Loss 2.417552947998047, Domain Loss 1.4498441219329834
Step 10
Class Loss 0.37697210907936096, Domain Loss 0.4205854833126068


100%|██████████| 16/16 [00:04<00:00,  3.31it/s]


Accuracy on test set: 0.400390625
Starting epoch 2/10, LR = [0.001]
Step 20
Class Loss 0.173111230134964, Domain Loss 0.31948065757751465


...

100%|██████████| 16/16 [00:04<00:00,  3.44it/s]

Accuracy on test set: 0.4482421875
Starting epoch 10/10, LR = [6.25e-05]


Step 120
Class Loss 0.07040602713823318, Domain Loss 0.3593199849128723


100%|██████████| 16/16 [00:04<00:00,  3.38it/s]

Accuracy on test set: 0.44580078125
Photo transfer to Art accuracy DANN: 0.45654296875
********* FILE SAVED *********
